In [2]:
import librosa
import numpy as np
import scipy.linalg
from typing import List
from scipy.stats import zscore
from dataclasses import dataclass
import pandas as pd
from os import listdir
import mir_eval
import csv
import template
import pretty_midi
import scipy

@dataclass
class KeyEstimator:
    tM =  [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
    tm = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]

    def __post_init__(self):
        self.tM = zscore(self.tM)
        self.tM_norm = scipy.linalg.norm(self.tM)
        self.tM = scipy.linalg.circulant(self.tM)

        self.tm = zscore(self.tm)
        self.tm_norm = scipy.linalg.norm(self.tm)
        self.tm = scipy.linalg.circulant(self.tm)
    def __call__(self,x:np.array) -> List[np.array]:
        x = zscore(x)
        x_norm = scipy.linalg.norm(x)
        
        coeffs_tM = self.tM.T.dot(x) / self.tM_norm / x_norm
        coeffs_tm = self.tm.T.dot(x) / self.tm_norm / x_norm
        return coeffs_tM, coeffs_tm

def WA(ans, preds):
    new_accuracy = 0

    pr = preds
    la = ans
    if pr == la:
        new_accuracy += 1.
    if pr < 12 and la < 12:
        if pr == (la + 7) % 12:
            new_accuracy += 0.5
    elif pr >= 12 and la >= 12:
        pr -= 12
        la -= 12
        if pr == (la + 7) % 12:
            new_accuracy += 0.5

    # Relative major/minor
    if pr < 12 <= la:
        la -= 12
        if pr == (la + 3) % 12:
            new_accuracy += 0.3
    elif pr >= 12 and la < 12:
        pr -= 12
        if ((pr + 3) % 12 == la):
            new_accuracy += 0.3

    # Parallel major/minor
    if pr == (la + 12) % 24:
        new_accuracy += 0.2

    return new_accuracy


def main():
    CSV = pd.read_csv('./ann_audio_globalkey_2.csv', delimiter=';') 
    s = CSV.loc[:, "WorkID"]
    musicDict = {}
    acc_stft  =0
    acc_cqt   =0
    acc_cens  =0
    wacc_stft =0
    wacc_cqt  =0
    wacc_cens =0
    for i in range(s.shape[0]):
        song_id = s[i].split('_')[1].split(';')[0]
        instrument = s[i].split(';')[1].replace('"', '')
        key = s[i].split(';')[2].replace('"', '')
        if instrument != 'SC06':
            continue
        musicDict[song_id] = key
    folder_path ="SC06"
    files = listdir(folder_path)
    
    # file =input("./Schubert_D911-01_SC06.wav")
    for  file_id, f in enumerate(files):
        name=f
        new_name=name.split('_')[1]
        y,sr = librosa.load('{folder}/{file}'.format(folder = folder_path, file = f))
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
        num_frames = chroma_stft
        
        pitch_class_distribution_stft = chroma_stft.sum(axis=1)
        pitch_class_distribution_cqt = chroma_cqt.sum(axis=1)
        pitch_class_distribution_cens = chroma_cens.sum(axis=1)
        
        key_estimator = KeyEstimator()
        tM_stft, tm_stft =key_estimator(pitch_class_distribution_stft)
        tM_cqt, tm_cqt =key_estimator(pitch_class_distribution_cqt)
        tM_cens, tm_cens =key_estimator(pitch_class_distribution_cens)
        mapping ={0: "C:", 1: "Db:", 2: "D:", 3: "D#:", 4: "E:", 5: "F:", 6: "Gb:", 7: "G:", 8: "Ab:", 9: "A:", 10: "Bb:", 11: "B:"}
        inverse_mapping ={"C": 0, "Db": 1, "D": 2, "D#": 3,"Eb":3, "E": 4, "F": 5, "Gb": 6, "G": 7, "Ab": 8, "A": 9, "Bb": 10, "B": 11}
        ans_key=musicDict[new_name].split(':')[0]
        if np.max(tM_stft) > np.max(tm_stft):
            pred_key_stft =np.argmax(tM_stft)
        else:
            pred_key_stft =np.argmax(tm_stft)
        if np.max(tM_cqt) > np.max(tm_cqt):
            pred_key_cqt =np.argmax(tM_cqt)
        else:
            pred_key_cqt =np.argmax(tm_cqt)
        if np.max(tM_cens) > np.max(tm_cens):
            pred_key_cens =np.argmax(tM_cens)
        else:
            pred_key_cens =np.argmax(tm_cens)

        
  
        x_stft=mapping[np.argmax(tM_stft)] + "maj" if np.max(tM_stft) > np.max(tm_stft) else mapping[np.argmax(tm_stft)] + "min"
        x_cqt=mapping[np.argmax(tM_cqt)] + "maj" if np.max(tM_cqt) > np.max(tm_cqt) else mapping[np.argmax(tm_cqt)] + "min"
        x_cens=mapping[np.argmax(tM_cens)] + "maj" if np.max(tM_cens) > np.max(tm_cens) else mapping[np.argmax(tm_cens)] + "min"
        # print("ans:",inverse_mapping[ans_key])
        if (x_stft== musicDict[new_name]):
            acc_stft+=1
            
        wacc_stft+= WA(inverse_mapping[ans_key],pred_key_stft)
        if(x_cqt== musicDict[new_name]):
            acc_cqt+=1
           
        wacc_cqt+= WA(inverse_mapping[ans_key],pred_key_cqt)
        if(x_cens==musicDict[new_name]):
            acc_cens+=1
            
        wacc_cens+= WA(inverse_mapping[ans_key],pred_key_cens)
        
    RA_stft =  acc_stft/24
    RA_cqt  =  acc_cqt/24
    RA_cens  =  acc_cens/24
    wacc_stft = wacc_stft/24
    wacc_cqt  = wacc_cqt/24
    wacc_cens = wacc_cens/24
    print("RA_stft:",RA_stft)
    print("WA_stft:",wacc_stft)

    print("RA_cqt:",RA_cqt)
    print("WA_cqt:",wacc_cqt)

    print("RA_cens:",RA_cens)
    print("WA_cens:",wacc_cens)



















if __name__ == "__main__":

    main()
    
    


RA_stft: 0.5833333333333334
WA_stft: 0.7916666666666666
RA_cqt: 0.5416666666666666
WA_cqt: 0.8125
RA_cens: 0.5
WA_cens: 0.75


In [3]:
import csv
from os import listdir

localkey_folder = "ann_audio_localkey-ann1"
localkey_files = listdir(localkey_folder)
destination_folder = "ann_score_localkey-time-ann1"
flat2sharp = {'Cb':'B', 'Db':'C#', 'Eb':'D#', 'Fb':'E', 'Gb':'F#', 'Ab':'G#', 'Bb':'A#', 'min':'minor', 'maj':'major'}
for file in localkey_files:
    
    rows = []
    # version = file.split('_')[2]
    # if version not in [v + '.csv' for v in versions]:
    #     continue
    # version = version.split('.')[0]

    with open('{}/{}'.format(localkey_folder, file)) as f:
        # 1.000;15.750;"D:min"
        csvreader = csv.reader(f)
        header = next(csvreader)
       
        print(header)
        for row in csvreader:
            newrow = row[0].split(';')
            # print(newrow)
            newrow[2] = newrow[2].split('"')[1].split(':')
            
            if newrow[2][0][-1] == 'b':
                newrow[2] = flat2sharp[newrow[2][0]] + ' ' + flat2sharp[newrow[2][1]]
            else:
                newrow[2] = newrow[2][0] + ' ' + flat2sharp[newrow[2][1]]
            rows.append(newrow)
            print(rows)
    with open('{}/{}'.format(destination_folder, file), 'w', newline="") as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(rows)

['start;end;key']
[['0.24', '33.68', 'C minor']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major'], ['53', '107.72', 'C minor']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major'], ['53', '107.72', 'C minor'], ['107.72', '117.42', 'D# major']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major'], ['53', '107.72', 'C minor'], ['107.72', '117.42', 'D# major'], ['117.42', '126.78', 'G# major']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major'], ['53', '107.72', 'C minor'], ['107.72', '117.42', 'D# major'], ['117.42', '126.78', 'G# major'], ['126.78', '160.56', 'C minor']]
[['0.24', '33.68', 'C minor'], ['33.68', '43.4', 'D# major'], ['43.4', '53', 'G# major'], ['53', '107.72', 'C minor'], [

In [20]:
tonic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'major', 'minor']
KS_major_template = [[6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], 
                     [2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29],
                     [2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66], 
                     [3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39],
                     [2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19], 
                     [5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52],
                     [2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09], 
                     [4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38],
                     [4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33], 
                     [2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48],
                     [3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23], 
                     [2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35]]

KS_minor_template = [[6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], 
                     [3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34],
                     [3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69], 
                     [2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98],
                     [3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75], 
                     [4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54],
                     [2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53], 
                     [3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60],
                     [2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38], 
                     [5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52],
                     [3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68], 
                     [2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33]]
scores = []
raw_scores = []

time_window = 150   # 0.1 sec

folder_path = "SC06"
files = listdir(folder_path)

score = 0
raw_score = 0

num_of_frame = 0

ann1_folder = 'ann_score_localkey-time-ann1'
ann1_files = listdir(ann1_folder)
ans_list = []

for f in ann1_files:
        
    ans = []

    with open('{folder}/{file}'.format(folder = ann1_folder, file = f)) as anscsv:

        csvreader = csv.reader(anscsv)
        csv_list = list(csvreader)
            
        header = []
            
        for row_id, row in enumerate(csv_list):
                
            try:
                start_sec = int(row[0])
            except:
                start_sec = 10 * int(row[0].split('.')[0]) + int(row[0].split('.')[1][0])
                if len(row[0].split('.')[1]) > 1:
                    start_sec += 1
            
            if len(header) == 0:
                header.append(start_sec)

            if row_id == len(csv_list) - 1:
                try:
                    end_sec = int(row[1])
                except:
                    end_sec = 10 * int(row[1].split('.')[0]) + int(row[1].split('.')[1][0])   
                header.append(end_sec)

            ans.append([start_sec, row[2]])

        ans.insert(0, header)
        ans_list.append(ans)

    # print(ans_list) 

for file_id, filename in enumerate(files):
        
    ans_song_list = ans_list[file_id]
    ans_cur = 1
    cur_start, cur_end = ans_song_list[0][0], ans_song_list[0][1]

        # print('start: {}, end: {}'.format(cur_start, cur_end))
        # print(ans_song_list)

    y,sr = librosa.load('{folder}/{file}'.format(folder = folder_path, file = filename))
    
    f  = librosa.feature.chroma_stft(y=y, sr=sr)

    num_bin, num_frame = f.shape[0], min(f.shape[1], cur_end - cur_start + 1)
    
    for frame in range(cur_start, min(cur_end + 1, cur_start + num_frame)):
                
            # print('frame:', frame)
                
        num_of_frame += 1
                
        bin_avg = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                
        if frame >= time_window:
            frame_start = frame - time_window
        else:
            frame_start = 0

        if frame < f.shape[1] - time_window:
            frame_end = frame + time_window
        else:
            frame_end = f.shape[1] - 1
                

        for i in range(frame_start, frame_end + 1):
            for j in range(num_bin):
                bin_avg[j] += f[j][i]

        max_bin, max_r = 0, 0
        
        tonic_str = ''
        for i,ks in enumerate(KS_major_template):
            r = scipy.stats.pearsonr(bin_avg, ks)[0]
            if r > max_r or i == 0:
                max_r = r
                tonic_str = tonic[i] + ' ' + tonic[-2]

        for i,ks in enumerate(KS_minor_template):
            r = scipy.stats.pearsonr(bin_avg, ks)[0]
            if r > max_r:
                max_r = r
                tonic_str = tonic[i] + ' ' + tonic[-1]
                    
        if ans_cur < len(ans_song_list) - 1:
            if ans_song_list[ans_cur + 1][0] <= frame:
                ans_cur += 1

        answer_str = ans_song_list[ans_cur][1]
                
        score += mir_eval.key.weighted_score(answer_str, tonic_str)
        if answer_str == tonic_str:
            raw_score += 1
                
   
print('wei score: {:.6f}'.format(score/num_of_frame))
print('raw score: {:.6f}'.format(raw_score/num_of_frame))

wei score: 0.369474
raw score: 0.272746


In [19]:
import pretty_midi
import mir_eval
import scipy
import numpy as np
from os import listdir
import template
import csv

scores = []
raw_scores = []
tonic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'major', 'minor']
KS_major_template = [[6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], 
                     [2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29],
                     [2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66], 
                     [3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39],
                     [2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19], 
                     [5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52],
                     [2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38, 4.09], 
                     [4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33, 4.38],
                     [4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48, 2.33], 
                     [2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23, 3.48],
                     [3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35, 2.23], 
                     [2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.28, 6.35]]

KS_minor_template = [[6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], 
                     [3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34],
                     [3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69], 
                     [2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98],
                     [3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75], 
                     [4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54],
                     [2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60, 3.53], 
                     [3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38, 2.60],
                     [2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52, 5.38], 
                     [5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68, 3.52],
                     [3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33, 2.68], 
                     [2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17, 6.33]]
time_window = 150   # 0.1 sec

inverse_mapping ={"C": 0, "Db": 1, "D": 2, "D#": 3,"Eb":3, "E": 4, "F": 5, "Gb": 6, "G": 7, "Ab": 8, "A": 9, "Bb": 10, "B": 11}
folder_path = "SC06"
files = listdir(folder_path)

score = 0
raw_score = 0

num_of_frame = 0

ann1_folder = "ann_score_localkey-time-ann1"
ann1_files = listdir(ann1_folder)
ans_list = []
key_list = []
second=0
for f in ann1_files:
        
    ans = []
    Dict={}
    with open('{folder}/{file}'.format(folder = ann1_folder, file = f)) as anscsv:

        csvreader = csv.reader(anscsv)
        csv_list = list(csvreader)
        header = []  
       
        
        for row_id, row in enumerate(csv_list):
            

            try:
                start_sec = (int(row[0]))
            except:
                # start_sec = 10 * int(row[0].split('.')[0]) + int(row[0].split('.')[1][0])

                # if len(row[0].split('.')[1]) > 1:
                start_sec = (int(row[0].split('.')[0]))
                
            
            if row_id == 0:
                header.append(start_sec)
            try:
                end_sec = int(row[1]) 
            except:
                end_sec = int(row[1].split('.')[0])
            # end_sec = 10 * int(row[1].split('.')[0]) + int(row[1].split('.')[1][0])
            if row_id == len(csv_list) - 1:
                header.append(end_sec)
            

            key = row[2]   
            # print("end:",end_sec)
            
            
            

            ans.append([start_sec,end_sec])
            # print("start_sec:",start_sec)
            # print("end_sec:",end_sec)
            # print("row[0].split('.')[0]:",row[0].split('.')[0])
            # print("int(row[0].split('.')[1][0] :",int(row[0].split('.')[1][0]))
        ans.insert(0, header)
        ans_list.append(ans)
            

for file_id, filename in enumerate(files):
        
    # ans_song_list = ans_list[file_id][1:]
    
    ans_song_list = ans_list[file_id][1:]
    ans_cur = 1
    cur_start, cur_end = ans_song_list[0][0], ans_song_list[0][1]
    # print('start, end:', cur_start, ',',cur_end)
    estimate_list = []
    y,sr = librosa.load('{folder}/{file}'.format(folder = folder_path, file = filename))
    chroma_stft  = librosa.feature.chroma_stft(y=y, sr=sr)

    num_bin, num_frame = chroma_stft.shape[0], min(chroma_stft.shape[1], cur_end - cur_start + 1)

    last_tonic_str = '' 
    seg_start, seg_end = cur_start, cur_start

    for frame in range(cur_start, min(cur_end + 1, cur_start + num_frame)):
        
        num_of_frame += 1
                
        bin_avg = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                
        if frame >= time_window:
            frame_start = frame - time_window
        else:
            frame_start = 0

        if frame < chroma_stft.shape[1] - time_window:
            frame_end = frame + time_window
        else:
            frame_end = chroma_stft.shape[1] - 1
                
       
        for i in range(frame_start, frame_end + 1):
            for j in range(num_bin):
                bin_avg[j] += chroma_stft[j][i]
        
        max_bin, max_r = 0, 0

       
        tonic_str = ''
        for i,ks in enumerate(KS_major_template):
            r = scipy.stats.pearsonr(bin_avg, ks)[0]
            if r > max_r or i == 0:
                max_r = r
                tonic_str = tonic[i] + ' ' + tonic[-2]

        for i,ks in enumerate(KS_minor_template):
            r = scipy.stats.pearsonr(bin_avg, ks)[0]
            if r > max_r:
                max_r = r
                tonic_str = tonic[i] + ' ' + tonic[-1]
        if ans_cur < len(ans_song_list) - 1:
            if ans_song_list[ans_cur + 1][0] <= frame:
                ans_cur += 1
 
    
        
        

        if frame == cur_start:
            last_tonic_str = tonic_str
        
        if tonic_str != last_tonic_str:
            seg_end = frame
            estimate_list.append([seg_start/10, seg_end/10])
            seg_start = seg_end + 1
        
        last_tonic_str = tonic_str
    
    if seg_start < min(cur_end + 1, cur_start + num_frame) - 1:
        estimate_list.append([seg_start/10, (min(cur_end + 1, cur_start + num_frame) - 1)/10])
    
    score_u = mir_eval.chord.underseg(np.array(ans_song_list), np.array(estimate_list))
    score_o = mir_eval.chord.overseg(np.array(ans_song_list), np.array(estimate_list))
    score_a = mir_eval.chord.seg(np.array(ans_song_list), np.array(estimate_list))
    
    print('| file{:02d} | {} |'.format(file_id, filename))
    print('| ---- | ---- |')
    print('| underseg | {} |'.format(score_u))
    print('| overseg | {} |'.format(score_o))
    print('| meanseg | {} |'.format(score_a))
    print()


| file00 | Schubert_D911-01_SC06.wav |
| ---- | ---- |
| underseg | 1.0 |
| overseg | 0.9898148148148148 |
| meanseg | 0.9898148148148148 |

| file01 | Schubert_D911-02_SC06.wav |
| ---- | ---- |
| underseg | 1.0 |
| overseg | 0.9821052631578947 |
| meanseg | 0.9821052631578947 |

| file02 | Schubert_D911-03_SC06.wav |
| ---- | ---- |
| underseg | 1.0 |
| overseg | 0.9735294117647059 |
| meanseg | 0.9735294117647059 |

| file03 | Schubert_D911-04_SC06.wav |
| ---- | ---- |
| underseg | 1.0 |
| overseg | 0.9823899371069182 |
| meanseg | 0.9823899371069182 |

| file04 | Schubert_D911-05_SC06.wav |
| ---- | ---- |
| underseg | 0.8928571428571428 |
| overseg | 0.9738675958188153 |
| meanseg | 0.8928571428571428 |

| file05 | Schubert_D911-06_SC06.wav |
| ---- | ---- |
| underseg | 0.859375 |
| overseg | 0.9769874476987448 |
| meanseg | 0.859375 |

| file06 | Schubert_D911-07_SC06.wav |
| ---- | ---- |
| underseg | 1.0 |
| overseg | 0.9894736842105263 |
| meanseg | 0.9894736842105263 |

| f